In [ ]:
import openai
from importlib import reload
from utils.structure_content import process_folder_concurrently,StructuredDocument
from llm_client import llm_client
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    Document,
    DocumentStore,
    SimpleDocument,
    VectorStore
)
from kme_doc import KMEDocument
from tqdm import tqdm
import pandas as pd
import os
from config.settings import Settings
from content_retrieval import query_document, llm,doc_store, summary_doc_store ,embed,embedding
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore

settings = Settings()

2025-09-11 10:33:42,099 - INFO - Loading faiss with AVX512 support.
2025-09-11 10:33:42,099 - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-09-11 10:33:42,100 - INFO - Loading faiss with AVX2 support.
2025-09-11 10:33:42,210 - INFO - Successfully loaded faiss with AVX2 support.


Loaded FAISS index (358 vectors) and ID set from disk.
Syncing VectorStore with DocumentStore...
Found 1 missing documents. Indexing in batches of 128...


2025-09-11 10:33:45,909 - INFO - Retrying request to /embeddings in 0.486265 seconds
2025-09-11 10:33:48,440 - INFO - Retrying request to /embeddings in 0.954407 seconds


APIConnectionError: Connection error.

In [2]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl").reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()
ob_documents = kme_documents.query("BELASTINGSOORT == 'OB - Omzetbelasting'")
results = add_new_documents_to_docstore(ob_documents,doc_store)

In [4]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

### Samenvatten en segmenteren content

In [5]:
stats = summarize_new_documents(
    doc_store=doc_store,
    summary_doc_store=summary_doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    KMEDocument_cls=KMEDocument,
    max_workers=10,
    start=0,
    count=1500,
    show_progress=True
)

Summarizing: 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]


In [2]:
import json
if os.path.exists(settings.data_root / "project_list.json"):
    with open(settings.data_root / "project_list.json") as f:
        project_list = json.load(f)

NameError: name 'settings' is not defined